<a href="https://colab.research.google.com/github/mikenlee/GWU_dataAnalytics_HW/blob/main/housing_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Introduction**
This project explores a 2018 national dataset of individual mortgage originations from the Federal Housing Finance Finance Agency (FHFA). The dataset captures demographic information (i.e.,race, gender, median income in a locale) as well as mortgage related information (i.e., loan to value, income ratio, dwelling type)

In [2]:
# install python modules
!pip install tabula-py

#import dependencies
import pandas as pd
import tabula
import os

     |████████████████████████████████| 11.7MB 331kB/s 


In [3]:
#activate R magic
%load_ext rpy2.ipython

/usr/local/lib/python3.6/dist-packages/rpy2/robjects/pandas2ri.py:14: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import Index as PandasIndex
/usr/local/lib/python3.6/dist-packages/rpy2/robjects/pandas2ri.py:34: UserWarning: pandas >= 1.0 is not supported.
  warnings.warn('pandas >= 1.0 is not supported.')


In [4]:
%%R
getwd()

[1] "/content"


In [5]:
# install R packages
%%R
install.packages("tidyverse")

R[write to console]: Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

R[write to console]: trying URL 'https://cran.rstudio.com/src/contrib/tidyverse_1.3.0.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 712837 bytes (696 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[wr

In [6]:
# load R packages
%%R
library(tidyverse)

R[write to console]: ── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

R[write to console]: ✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.4     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.0

R[write to console]: ── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



In [7]:
#mount your google drive giving access to your files in colab
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


#**Create Data Dictionary from a PDF Table**

In [8]:
#create DF from tables in the PDF dictionary
dict_df = pd.DataFrame(columns = [0,1,2])
for page in range(1,6):
    pdf_table = tabula.read_pdf("/content/drive/My Drive/GW_project1/data/dict/AMA_PUDB_definitions_2018.pdf", pages = page)
    pdfDF = pdf_table[0].T.reset_index().T
    dict_df = pd.concat([dict_df, pdfDF], axis = 0, ignore_index = True)
dict_df

,0,1,2
0,1,Year,Year Loan was reported
1,2,Assigned ID,Unique Record ID (Not Actual Loan Number)
2,3,Bank,Name of Federal Home Loan Bank District
3,4,FIPSStateCode,FIPS State Code
4,5,FIPSCountyCode,FIPS County Code
...,...,...,...
260,NaN,NaN,8 - Risk Sharing
261,NaN,NaN,9 - REIT
262,82,GSEREO,GSE Real Estate Owned
263,NaN,NaN,1 – Yes


In [9]:
# save as csv and move to google drive
dict_df.to_csv('dict2018.csv')
!cp dict2018.csv "/content/drive/My Drive/GW_project1/data/dict/"

In [10]:
%%R -o pdf_tib

#read in data from google drive
dictDF = read_csv("/content/drive/My Drive/GW_project1/data/dict/dict2018.csv")
fips_state = read_csv("/content/drive/My Drive/GW_project1/data/dict/fips_state_codes.csv")
fips_county = read_csv("/content/drive/My Drive/GW_project1/data/dict/fips_county_codes.csv")
#get MSA data

#clean data
codeDF = dictDF %>%
  select(`1`, `2`) %>%
  rename(orig_col = `1`) %>%
  separate(`2`, c('description', 'value'), sep = c("=|\\s\\--\\s|\\s\\-\\s|\\s\\–\\s")) %>% #separate on delimiters
  mutate(code = ifelse(is.na(value), NA, description)) %>% # new col for codes
  mutate(description = ifelse(is.na(orig_col), NA, description)) %>% #make str codes NA to fill next
  fill(description) %>%
  fill(orig_col) %>%
  mutate(value = str_remove_all(value, ";"))

# add FIPS state data
code_state = fips_state %>%
  rename(value = `Postal Code`,
         description = Name,
         code = FIPS) %>%
  mutate(orig_col = "FIPSStateCode",
         code = as.character(code)) %>%
  bind_rows(c('description' = 'District of Columbia', 'value' = 'DC', 'code' = '11')) %>%  
  bind_rows(codeDF)

# add FIPS county data
pdf_tib = fips_county %>%
  rename(value = Name,
         description = State,
         code = FIPS) %>%
  mutate(orig_col = "FIPSCountyCode",
         code = as.character(code),
         code = ifelse(str_length(code) == 4, paste('0', code, sep = ''), code)) %>%
  bind_rows(code_state) %>%
  arrange(orig_col) %>%
  select(orig_col, description, code, value)


R[write to console]: 
── Column specification ────────────────────────────────────────────────────────
cols(
  X1 = col_double(),
  `0` = col_double(),
  `1` = col_character(),
  `2` = col_character()
)


R[write to console]: 
── Column specification ────────────────────────────────────────────────────────
cols(
  Name = col_character(),
  `Postal Code` = col_character(),
  FIPS = col_double()
)


R[write to console]: 
── Column specification ────────────────────────────────────────────────────────
cols(
  FIPS = col_double(),
  Name = col_character(),
  State = col_character()
)




In [11]:
# convert pdf tibble to a nested dictionary
from collections import defaultdict

d = defaultdict(dict)

for row in pdf_tib.itertuples(index=False):
    d[row.orig_col][row.code] = row.value

pdf_dict = dict(d)

#**Clean in 2018 data**

In [12]:
%%R -o fhfa18_tib2
# read in data and clean
fhfa18_tib = read_csv("/content/drive/My Drive/GW_project1/data/2018_PUDB_EXPORT_123118.csv")

fhfa18_tib2 = fhfa18_tib %>%
  mutate_if(is.double, as.integer) %>% #change to integer first to get rid of the float
  mutate_if(Negate(is.character), as.character) %>% 
  mutate(FIPSCountyCode = ifelse(str_length(FIPSCountyCode) == 2, 
                                 paste('0',FIPSCountyCode, sep=''), 
                                 ifelse(str_length(FIPSCountyCode) == 1,
                                        paste('00', FIPSCountyCode, sep=''), FIPSCountyCode)),
         FIPSCountyCode = ifelse(str_length(FIPSStateCode) < 2, 
                                 paste('0', FIPSStateCode, FIPSCountyCode, sep=''), 
                                 paste(FIPSStateCode, FIPSCountyCode, sep=''))) #mutate county code format to match dictionary


R[write to console]: 
── Column specification ────────────────────────────────────────────────────────
cols(
  .default = col_double(),
  Bank = col_character(),
  PropType = col_character(),
  PrepayP = col_character()
)
ℹ Use `spec()` for the full column specifications.




In [1]:
# assign DF to a new variable 
fhfa18 = pd.DataFrame()
fhfa18 = fhfa18_tib2.copy() #assign independent copy so changes in one DF doesn't affect the other DF

NameError: ignored

In [46]:
# replace values in data columns with dictionary values (need to clean dictionary)
selected_cols = ['FIPSStateCode', 'FIPSCountyCode', 'Purpose', 'First', 'BoRace', 'BoGender', 'Geog', 'Occup', 'PropType']
# selected_cols = ['FIPSStateCode', 'FIPSCountyCode']
# selected_cols = ['FIPSStateCode']

for col in selected_cols:
    for i, row in enumerate(fhfa18[col]):
      try:
        print(fhfa18.loc[str(i+1),col])
        fhfa18.loc[str(i+1),col] = pdf_dict[col][row]

        print(pdf_dict[col][row])
        print(fhfa18.loc[str(i+1),col])
        print(f' - - - - {i+1}')
      except KeyError:
        print(f"error {fhfa18.loc[str(i+1),col]}- - - {i+1} -- {pdf_dict[col][row]}")
        
    if fhfa18.loc[str(i+1),col] != pdf_dict[col][row]:
      print(f"ERROR {i+1} {col} {row}")
      break
    else:
      continue

24
MD
MD
 - - - - 1
24
MD
MD
 - - - - 2
51
VA
VA
 - - - - 3
51
VA
VA
 - - - - 4
24
MD
MD
 - - - - 5
24
MD
MD
 - - - - 6
51
VA
VA
 - - - - 7
1
AL
AL
 - - - - 8
45
SC
SC
 - - - - 9
45
SC
SC
 - - - - 10
45
SC
SC
 - - - - 11
45
SC
SC
 - - - - 12
8
CO
CO
 - - - - 13
51
VA
VA
 - - - - 14
45
SC
SC
 - - - - 15
42
PA
PA
 - - - - 16
24
MD
MD
 - - - - 17
24
MD
MD
 - - - - 18
51
VA
VA
 - - - - 19
45
SC
SC
 - - - - 20
51
VA
VA
 - - - - 21
13
GA
GA
 - - - - 22
12
FL
FL
 - - - - 23
66
GU
GU
 - - - - 24
72
PR
PR
 - - - - 25
12
FL
FL
 - - - - 26
48
TX
TX
 - - - - 27
21
KY
KY
 - - - - 28
51
VA
VA
 - - - - 29
24
MD
MD
 - - - - 30
53
WA
WA
 - - - - 31
37
NC
NC
 - - - - 32
51
VA
VA
 - - - - 33
12
FL
FL
 - - - - 34
24
MD
MD
 - - - - 35
51
VA
VA
 - - - - 36
48
TX
TX
 - - - - 37
26
MI
MI
 - - - - 38
66
GU
GU
 - - - - 39
51
VA
VA
 - - - - 40
51
VA
VA
 - - - - 41
72
PR
PR
 - - - - 42
11


KeyError: ignored

#**Analysis**

In [89]:

fhfa18['LTV'] = fhfa18['LTV'].astype(float)
fhfa18['Income'] = fhfa18['Income'].astype(float)
fhfa18.groupby(["FIPSStateCode", "BoRace"])["Income"].mean()
fhfa18['FIPSStateCode'].unique()

array(['MD', 'VA', 'AL', 'SC', 'CO', 'PA', 'GA', 'FL', 'GU', 'PR', 'TX',
       'KY', 'WA', 'NC', 'MI', '11', 'MN', 'DE', 'IN', 'IL', 'TN', 'KS',
       'NV', 'AZ', 'MA', 'OR', 'NY', 'SD', 'NJ', 'OK', 'IA', 'ND', 'NH',
       'AK', 'OH', 'ID', 'LA', 'UT', 'NM', 'MS', 'WV', 'MT', 'CA', 'AR',
       'MO', 'WI', 'NE', 'CT', 'ME', 'VT', 'RI', 'HI', 'WY'], dtype=object)

In [47]:
fhfa18.query("FIPSStateCode == '10'")
fhfa18['FIPSStateCode'].unique()
fhfa18.loc["2547"]
fhfa18.loc['2','FIPSCountyCode']
fhfa18.query("FIPSStateCode == '1'")
fhfa18.loc['70','FIPSStateCode']

'1'

In [97]:
pdf_dict['FIPSCountyCode']['200113']

KeyError: ignored

In [88]:
fhfa18



,Year,AssignedID,Bank,FIPSStateCode,FIPSCountyCode,MSA,FeatureID,Tract,MinPer,TraMedY,LocMedY,Tractrat,Income,CurAreY,IncRat,UPB,LTV,MortDate,AcquDate,Purpose,Coop,Product,FedGuar,Term,AmorTerm,SellType,NumBor,First,CICA,BoRace,CoRace,BoGender,CoGender,BoAge,CoAge,Occup,NumUnits,Bed1,Bed2,Bed3,...,Aff2,Aff3,Aff4,Rent1,Rent2,Rent3,Rent4,RentUt1,RentUt2,RentUt3,RentUt4,Geog,Rate,Amount,Front,Back,BoCreditScore,CoBoCreditScore,PMI,Self,PropType,ArmIndex,ArmMarg,PrepayP,BoEth,Race2,Race3,Race4,Race5,CoEth,Corace2,Corace3,Corace4,Corace5,HOEPA,LienStatus,SpcHsgGoals,FedFinStbltyPlan,AcqTyp,GSEREO
1,2018,2131406,Atlanta,MD,24021,47900,9999999999,7512.01,29.85,93261,108141,0.8624,146196.0,110300,1.3254,463939,1.03,2017,2018,1,2,1,2,360,360,1,2,1,9,7,7,2,1,41,43,1,1,98,98,98,...,2,2,2,9999999999,9999999999,9999999999,9999999999,9999999999,9999999999,9999999999,9999999999,2,0.04,464782,0.2424,0.3221,4,4,0,2,PT07,99,99999,12/31/9999,3,6,6,6,6,3,6,6,6,6,2,1,2,2,1,2
2,2018,2131407,Atlanta,MD,24017,47900,9999999999,8509.04,67.24,112623,108141,1.0414,156972.0,110300,1.4231,495000,1.00,2017,2018,1,2,1,2,360,360,1,2,1,9,3,3,1,2,61,54,1,1,98,98,98,...,2,2,2,9999999999,9999999999,9999999999,9999999999,9999999999,9999999999,9999999999,9999999999,2,0.0375,495000,0.2384,0.455,4,5,0,2,PT07,99,99999,12/31/9999,2,6,6,6,6,2,6,6,6,6,2,1,2,2,1,2
3,2018,2131408,Atlanta,VA,510153,47900,9999999999,9012.36,40.76,140156,108141,1.296,337464.0,110300,3.0595,664307,0.99,2017,2018,2,2,1,2,360,360,1,2,2,9,3,3,1,2,55,61,1,1,98,98,98,...,2,2,2,9999999999,9999999999,9999999999,9999999999,9999999999,9999999999,9999999999,9999999999,2,0.0388,665287,0.1369,0.2102,5,5,0,2,PT07,99,99999,12/31/9999,2,6,6,6,6,2,6,6,6,6,2,1,2,2,1,2
4,2018,2131409,Atlanta,VA,51059,47900,9999999999,4221.02,75.97,89250,108141,0.8253,132000.0,110300,1.1967,490320,1.03,2017,2018,1,2,1,2,360,360,1,1,1,9,2,8,1,4,27,98,1,1,98,98,98,...,2,2,2,9999999999,9999999999,9999999999,9999999999,9999999999,9999999999,9999999999,9999999999,2,0.0375,490320,0.2582,0.2951,5,9,0,2,PT07,99,99999,12/31/9999,2,6,6,6,6,5,8,8,8,8,2,1,2,2,1,2
5,2018,2131410,Atlanta,MD,2405,12580,9999999999,4924.02,64.56,81397,87788,0.9272,29376.0,91100,0.3225,97206,0.99,2017,2018,1,2,1,1,360,360,1,1,1,9,3,8,1,4,22,98,1,1,98,98,98,...,2,2,2,9999999999,9999999999,9999999999,9999999999,9999999999,9999999999,9999999999,9999999999,2,0.0375,97206,0.4012,0.5304,4,9,0,2,PT01,99,99999,12/31/9999,2,6,6,6,6,5,8,8,8,8,2,1,2,2,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65699,2018,2161350,Topeka,OK,400143,46140,9999999999,75.22,22.82,69635,62478,1.1146,100008.0,67100,1.4904,195865,0.99,2018,2018,1,2,1,1,360,360,1,2,2,9,5,5,1,1,43,69,1,1,98,98,98,...,2,2,2,9999999999,9999999999,9999999999,9999999999,9999999999,9999999999,9999999999,9999999999,2,0.0525,195865,0.3263,0.4912,5,5,0,2,PT07,99,99999,12/31/9999,2,6,6,6,6,2,6,6,6,6,2,1,2,2,1,2
65700,2018,2161351,Topeka,NE,310109,30700,9999999999,37.09,7.61,94198,72700,1.2957,154452.0,78400,1.9701,296000,0.80,2018,2018,1,2,1,0,180,180,1,2,2,9,5,5,1,2,55,56,1,1,98,98,98,...,2,2,2,9999999999,9999999999,9999999999,9999999999,9999999999,9999999999,9999999999,9999999999,2,0.0438,296000,0.225,0.3657,3,4,0,2,PT07,99,99999,12/31/9999,2,6,6,6,6,2,6,6,6,6,2,1,2,2,1,2
65701,2018,2161352,Topeka,IA,190133,99999,9999999999,9602,3.69,69688,63366,1.0998,54540.0,67800,0.8044,66400,0.79,2018,2018,2,2,1,0,360,360,1,1,2,9,5,8,1,4,26,98,1,1,98,98,98,...,2,2,2,9999999999,9999999999,9999999999,9999999999,9999999999,9999999999,9999999999,9999999999,2,0.0475,66400,0.1089,0.2918,5,9,0,2,PT01,99,99999,12/31/9999,2,6,6,6,6,5,8,8,8,8,2,1,2,2,1,2
65702,2018,2161353,Topeka,NE,310153,36540,9999999999,106.23,10.16,98707,75898,1.3005,87996.0,81600,1.0784,24560

In [ ]:
%%R

fhfa18 %>%
  select(FIPSStateCode, FIPSCountyCode, LTV, MinPer, TraMedY, Income, Purpose, First, BoRace, BoGender, BoAge, Occup, PropType) %>%
  group_by(FIPSStateCode, BoRace) %>%
  summarize(mean(LTV))



#**Question to Explore:**

1. Distribution of home loans across demographic groups 
  - Are certain groups getting more home loans than others?
2. Variation across demographic groups in nonconventional loan use
  - Are minorities accessing various types of loans?
3. Distribution of affordable housing mortgages
  - Which states/regions experience more affordable housing purchases?
4. 

(https://files.consumerfinance.gov/f/documents/cfpb_2019-mortgage-market-activity-trends_report.pdf)



#**Lessons Learned About Python:**
1. If you assign a dataframe to another dataframe, changes in one will affect the other. Assigning an independent copy will prevent this. 

2. Printing an error message with variables in the for loop greatly helps in debugging. 

3. The Tidyverse package in R and Python's list comprehension is a powerful combination during data cleaning. 

#**Lessons Learned About the Data Analytics Workflow**
1. When there is neither a customer nor a driving business/analytical problem statement, just dig into what data you can find and see if you find something interesting. Your initial problem statement will change anyway. 

2. Time is the limiting factor in this class so dedicate your efforts to the aspect of data science that you want to learn. If you want to get better at data cleaning, focus on that. If you want to get better at charting, focus on that. 
